# MLDM Lab week 3: Neural Networks - Perceptron and Multi-Layer Perceptron (MLP)

<h3> <font color="blue"> Introduction </h3>

In this lab session, we explore Perceptron and Multi-Layer Perceptron (MLP) using `sklearn`.

We revisit the Iris and the Breast Cancer datasets from previous lab sessions and train Perceptron and MLP classifiers using these datasets. Please see the information regarding these datasets from previous two lab sessions.

<h3> <font color="blue"> Lab goals</font> </h3>
<p> 1.  Learn how to create and use a Perceptron. </p>
<p> 2.  Learn how to create and use a MLP. </p>
<p> 3.  Learn how to create ROC curves to evaluate and compare models. </p>

## <font color="blue"> Training and evaluating a Perceptron 
In this experiment we re-visit the Iris dataset, however, instead of loading the dataset from a file or url, we load the dataset directly from the scikit-learn datasets. In this dataset, the third column represents the petal length, and the fourth column the petal width of the flower examples and the classes are already converted to integer labels where 0=Iris-Setosa, 1=Iris-Versicolor, 2=Iris-Virginica.

In [ ]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target

print('Class labels:', np.unique(y))

Preparing the training and test data by splitting the main dataset into 70% training and 30% test stes:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

In [ ]:
print('Labels counts in y:', np.bincount(y))
print('Labels counts in y_train:', np.bincount(y_train))
print('Labels counts in y_test:', np.bincount(y_test))

The features can be standardised using the `StandardScaler`. See <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html">`class sklearn.preprocessing.StandardScaler`</a> for more details.

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

### <font color="blue"> Training a Perceptron 
    
We train a Perceptron on the Iris dataset. In the first experiment we use simple and basic parameters. 
    
More information about Perceptron parameters can be found from <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html"> `sklearn.linear_model.Perceptron` </a>.

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

ppn = Perceptron(eta0=0.1, random_state=1)
ppn.fit(X_train_std, y_train)

**Note** You can replace `Perceptron(n_iter, ...)` by `Perceptron(max_iter, ...)` in scikit-learn >= 0.19. The `n_iter` parameter is used here deriberately, because some people still use scikit-learn 0.18.

In [ ]:
y_pred = ppn.predict(X_test_std)
print('Misclassified examples: %d' % (y_test != y_pred).sum())

In [ ]:
print('Accuracy(test set): %.3f' % accuracy_score(y_test, y_pred))
print('Accuracy (standardised test set): %.3f' % ppn.score(X_test_std, y_test))

## <font color="blue"> Training and evaluating a Multi-Layer Perceptron (MLP)
In this experiment we re-visit the breast cancer dataset, however, instead of loading the dataset from scikit-learn datasets, we load the dataset from a csv file. Please download (and unzip) the breast cancer dataset from SurreyLearn and copy the file 'breast_cancer_data.csv' into your Jupyter working directory.

In [ ]:
import pandas as pd
breast_cancer = pd.read_csv('breast_cancer_data.csv')
# Dataset Shape
print (breast_cancer.shape)

In [ ]:
# First Few Columns and Head Details
breast_cancer.head()

<p> We need to specify the features which are used for machine learning. We need to remove the features which should be excluded, e.g. ID. We also need to specify the target feature, i.e. diagnosis class.
    Feature Engineering is the way of extracting features from data and transforming them into formats that are suitable for Machine Learning algorithms.</p> Here we are going to remove certain unwanted features.

In [ ]:
# Features "id" and "Unnamed: 32" should be removed
feature_names = breast_cancer.columns[2:-1]
X = breast_cancer[feature_names]
# the target feature, i.e. diagnosis class
y = breast_cancer.diagnosis

<p>The traget feature in this dataset is included as a text value so it should be converted into a numerical value.</p>

In [ ]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
# M -> 1 and B -> 0
y = class_le.fit_transform(breast_cancer.diagnosis.values)

In [ ]:
breast_cancer[feature_names]

### <font color="blue"> Training a Multi-Layer Perceptron (MLP)
We train a Multi-Layer Perceptron (MLP) on the breast cancer dataset. In the first experiment we use simple parameters which we will try to optimise later. 
    
More information about MLP parameters can be found from <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html"> `sklearn.neural_network.MLPClassifier` </a>.

In [ ]:
# Training and testing data preperation.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Initializing the classifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=0, activation='logistic', hidden_layer_sizes=(5,), solver='adam', max_iter=100)
clf.fit(X_train, y_train)
clf_predict = clf.predict(X_test)
# Accuracy factors
print('acc for training data: {:.3f}'.format(clf.score(X_train, y_train)))
print('acc for test data: {:.3f}'.format(clf.score(X_test, y_test)))
print('MLP Classification report:\n\n', classification_report(y_test, clf_predict))

We can improve the accuracy of the MLP by tuning the parameters, e.g. increasing the 'hidden_layer_sizes' or 'max_iter'. Below we increase hidden_layer_sizes from 5 to 10.

In [ ]:
clf = MLPClassifier(random_state=0, activation='logistic', solver='adam', hidden_layer_sizes=(10,), max_iter=100)
clf.fit(X_train, y_train)
clf_predict = clf.predict(X_test)

# Accuracy factors
print('acc for training data: {:.3f}'.format(clf.score(X_train, y_train)))
print('acc for test data: {:.3f}'.format(clf.score(X_test, y_test)))
print('MLP Classification report:\n\n', classification_report(y_test, clf_predict))

<h3><font color="red">Exercise 1 </font> </h3>
<p>Repeat the experiment above and try to further improve the accuracy of the MLP by increasing 'max_iter'</p>

<p>Use the code cell below to write your code for Excercise 1</p>

In [ ]:
# Answer to Exercise 1


<h3><font color="red">Exercise 2 </font> </h3>
<p>Repeat the experiment above using a Perceptron on the breast cancer dataset and compare the accuracy with the results from MLP.</p>

<p>Use the code cell below to write your code for Excercise 2</p>

In [ ]:
# Answer to Exercise 2


<h3> <font color="blue"> Receiver Operating Characteristic (ROC) curves </font> </h3>

ROC curves are one of the most useful ways to evaluate and compare learning algorithms. Below we compare the performance of two MLP classifiers (i.e. MLP1 and MLP2) with different parameters

In [ ]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# generate a random prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]
# fit first model(MLP 1)
clf1 = MLPClassifier(random_state=0, activation='logistic', hidden_layer_sizes=(5,), max_iter=100)
clf1.fit(X_train, y_train)
# fit second model(MLP 2)
clf2 = MLPClassifier(random_state=0, activation='logistic', hidden_layer_sizes=(10,), max_iter=100)
clf2.fit(X_train, y_train)


# predict probabilities for different models
lr_probs1 = clf1.predict_proba(X_test)
lr_probs2 = clf2.predict_proba(X_test)

# keep probabilities for the positive outcome only
lr_probs1 = lr_probs1[:, 1]
lr_probs2 = lr_probs2[:, 1]

# calculate accuracy score for random prediction model
ns_auc = roc_auc_score(y_test, ns_probs)

# calculate accuracy score different MLP models
lr_auc1 = roc_auc_score(y_test, lr_probs1)
lr_auc2 = roc_auc_score(y_test, lr_probs2)

# summarize scores
print('Baseline (random guess): ROC AUC=%.3f' % (ns_auc))
print('MLP 1 (hidden layer sizes=5): ROC AUC=%.3f' % (lr_auc1))
print('MLP 2 (hidden layer sizes=10): ROC AUC=%.3f' % (lr_auc2))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr1, lr_tpr1, _ = roc_curve(y_test, lr_probs1)
lr_fpr2, lr_tpr2, _ = roc_curve(y_test, lr_probs2)

# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='Baseline (random guess)')
pyplot.plot(lr_fpr1, lr_tpr1, marker='.', label='MLP 1 (hidden layer sizes=5)')
pyplot.plot(lr_fpr2, lr_tpr2, marker='.', label='MLP 2 (hidden layer sizes=10)')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()


<h3><font color="red">Exercise 3 </font> </h3>
<p>Repeat the experiment above and define a new MLP classifier (named MLP 3) try to further improve the accuracy of the MLP 3 by changing the parameters, e.g. increasing 'hidden_layer_sizes' and 'max_iter'. What is the max accuracy you can get by tuning the parameters ?</p>

<p>Use the code cell below to write your code for Excercise 3</p>

In [ ]:
# Answer to Exercise 3


<h3><font color="red">Exercise 4 </font> </h3>
<p>Repeat the experiment in Exercise 3 but instead of a new MLP classifier define a Perceptron (named P 1) on the same dataset (i.e. breast cancer) and compare the result with MLP 1 and MLP 2. What do you think might be the reason for the difference between the performance of the MLP classifiers and the Perceptron on this dataset? Try to chnage the Perceptron parameters (reduce the learning rate, Eta) and repeat the experiment. </p>

<p>Use the code cell below to write your code for Exercise 4</p>

In [ ]:
# Answer to Exercise 4 
# Hint: Unlike MLP, Perceptron doesn't have 'predict_proba' but you can implement it using CalibratedClassifierCV as shown below
#
# ppn = Perceptron(eta0=0.01, random_state=0, max_iter=100)
# ppn = CalibratedClassifierCV(ppn)
#
# and then you can call ppn.predict_proba